# System Prompt 개념과 활용

## 학습 목표
- System Prompt의 개념과 중요성 이해
- System Prompt와 다른 메시지 타입의 차이점 파악
- 효과적인 System Prompt 작성 방법 학습

## 1. System Prompt란?

### 정의
- **System Prompt**는 AI 모델의 전체적인 역할, 행동 방식, 답변 스타일을 정의하는 기본 지침입니다.
- 대화의 시작 전에 AI에게 "당신은 어떤 역할을 하고, 어떻게 행동해야 하는지"를 알려주는 설정입니다.

### 주요 특징
1. **전역적 영향**: 모든 대화에 걸쳐 일관되게 적용됩니다.
2. **우선순위**: Human/AI 메시지보다 높은 우선순위를 가집니다.
3. **지속성**: 대화 전체에서 유지되는 기본 규칙입니다.

### 메시지 타입 비교

| 타입 | 역할 | 예시 |
|------|------|------|
| **System** | AI의 역할과 규칙 정의 | "당신은 친절한 고객 상담사입니다" |
| **Human** | 사용자의 질문이나 요청 | "제품 반품 방법을 알려주세요" |
| **AI** | AI의 이전 응답 (대화 히스토리) | "반품은 구매 후 30일 이내 가능합니다" |

## 2. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키가 설정되어 있나요?: {openai_api_key[:10] if openai_api_key else 'No'}...")

In [ ]:
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

print("모델이 성공적으로 초기화되었습니다!")

## 3. System Prompt 없이 사용하기

먼저 System Prompt 없이 직접 질문을 해봅시다.

In [ ]:
# System Prompt 없이 직접 질문
response = llm.invoke("안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?")

print("[System Prompt 없음]")
print(response.content)

## 4. System Prompt 사용하기

이제 System Prompt를 추가하여 AI의 역할을 명확하게 정의해봅시다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# System Prompt를 포함한 ChatPromptTemplate 생성
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 전문 헬스 트레이너입니다. 
        항상 운동의 정확한 자세와 안전을 강조하며, 초보자도 이해하기 쉽게 설명합니다.
        - 한가지 운동에 대해 자세한 자세 가이드를 설명해주어야합니다.
        """),
        ("human", "{input}")
    ]
)

# 체인 생성
chain = chat_template | llm

# 동일한 질문 실행
response = chain.invoke({"input": "안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?"})

print("[System Prompt: 전문 헬스 트레이너]")
print(response.content)

### 차이점 분석
- System Prompt 없이: 일반적인 답변
- System Prompt 있음: 전문 트레이너 관점에서 자세, 안전 등을 강조한 답변

## 5. 실습 2: System Prompt의 구체성

System Prompt가 구체적일수록 더 정확한 응답을 얻을 수 있습니다.

In [ ]:
# 추상적인 System Prompt
abstract_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 도움이 되는 어시스턴트입니다."),
        ("human", "{input}")
    ]
)

# 구체적인 System Prompt
specific_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 고객 서비스 전문가입니다. 다음 규칙을 반드시 따라주세요:
1. 항상 존댓말을 사용합니다.
2. 답변은 3문장 이내로 간결하게 작성합니다.
3. 문제 해결책을 단계별로 제시합니다.
4. 추가 질문이 있는지 물어봅니다.
5. 긍정적이고 해결 지향적인 태도를 유지합니다."""),
        ("human", "{input}")
    ]
)

# 테스트 질문
test_question = "자전거 체인이 빠졌어요."

In [ ]:
# 추상적 System Prompt 결과
chain1 = abstract_template | llm
response1 = chain1.invoke({"input": test_question})

print("[추상적인 System Prompt]")
print(response1.content)
print("\n" + "="*80 + "\n")

In [ ]:

# 구체적 System Prompt 결과
chain2 = specific_template | llm
response2 = chain2.invoke({"input": test_question})

print("[구체적인 System Prompt]")
print(response2.content)
print("\n" + "="*80 + "\n")

### 핵심!!
- 구체적인 System Prompt는 일관된 형식과 톤을 유지합니다.
- 명확한 규칙은 AI의 행동을 예측 가능하게 만듭니다.

## 6. 실습 5: 대화 히스토리와 System Prompt

System Prompt는 대화 전체에 걸쳐 일관되게 적용됩니다.

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# System Prompt와 대화 히스토리를 포함한 템플릿
conversation_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
당신은 친절한 영어 학습 도우미입니다. 
- 모든 답변에서 핵심 단어의 영어 표현을 괄호 안에 함께 제공합니다.
- 초보자도 이해하기 쉽게 설명합니다.
- 격려와 칭찬을 아끼지 않습니다.
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

# 대화 히스토리 시뮬레이션
chat_history = [
    HumanMessage(content="안녕하세요! 영어 공부를 시작하고 싶어요."),
    AIMessage(content="안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?")
]

chain = conversation_template | llm

In [ ]:
# 첫 번째 질문
response1 = chain.invoke({
    "chat_history": chat_history,
    "input": "기본 문법부터 배우고 싶어요."
})

print("[대화 1]")
print(response1.content)

# 대화 히스토리 업데이트
chat_history.extend([
    HumanMessage(content="기본 문법부터 배우고 싶어요."),
    AIMessage(content=response1.content)
])



In [ ]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

In [ ]:
# 두 번째 질문
response2 = chain.invoke({
    "chat_history": chat_history,
    "input": "현재완료 시제가 뭔가요?"
})

print("[대화 2]")
print(response2.content)

In [ ]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

In [ ]:
# 여기에 과제를 수행해보세요

# 과제 1: 여러분이 만들고 싶은 봇을 프롬프트로 만들어보세요
news_summarizer_prompt = """

"""